# Extração dos dados

## Texto introdutório

O objetivo deste projeto é fazer um modelo preditivo que consiga avaliar quem estará no pódio do prêmio de MVP(Neste projeto considerei os 4 primeiros, mas a NBA considera os 3 primeiros assim como qualquer premiação) e quem será o MVP.

Para isso, irei extrair as estatísticas da temporada regular de todos os jogadores da temporada 1979-80 até a temporada 2020-2021 para fazer o aprendizado do modelo de ML.Neste scraping, extrairei os dados totais,os dados por jogo,os dados por 36 min, os dados por 100 posses ,is dados avançados e os dados do desempenho do time em que o jogador esteve na temporada.Os dados de 2021-2022 serão a entrada para avaliar se o meu modelo consegue acertar os 4 primeiros lugares do prêmio de mvp deste ano.

Além de todas essas colunas que serão geradas na extração,no modelo de dataset proposto por mim, coloco as colunas 'mvp_candidate', 'mvp_podium' e 'mvp_winner'.Estas colunas são valores boleanos para avaliar essas condições.Também coloco a coluna 'mvp_votes_share'(coluna que armazena número de pontos recebidos dividido por número de pontos máximo.Caso o jogador não seja candidato a mvp, esse valor é 0).

As colunas descritas nos parágrafos acima tem como objetivo tentar montar um dataset em que seja possível usar a regressão linear para fazer a predição.Uso como valor de "chave" nomejogador_temporada_time para conseguir diferenciar apropiadamente os jogadores entre si e os valores numéricos são todos fornecidos pelos dados estatísticos.O objetivo é tentar prever o dado numérico de 'mvp_votes_share' de forma que a ordem seja a mesma do prêmio.

Obs:Tenho impressão de que vai haver mvp_votes_share negativos mesmo com uma filtragem, mas espero que ao menos dê para acertar o pódio

Neste notebook extrairei os dados do site basketball reference(https://www.basketball-reference.com/) para criar um dataset a partir de dados da  temporada regular da NBA.

## Importações e variáveis padrão

In [1]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from urllib.error import URLError, HTTPError
import pandas as pd

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36'}

#Necessário para ter as siglas dos times da NBA
times = {'Los Angeles Lakers' : 'LAL',
        'Milwaukee Bucks' : 'MIL',
        'Boston Celtics' : 'BOS',
        'Philadelphia 76ers' : 'PHI',
        'Portland Trail Blazers' : 'POR',
        'Detroit Pistons' : 'DET',
        'Denver Nuggets' : 'DEN',
        'Dallas Mavericks' : 'DAL',
        'Houston Rockets' : 'HOU',
        'San Antonio Spurs' : 'SAS',
        'New Jersey Nets' : 'NJN',
        'Utah Jazz' : 'UTA',
        'Golden State Warriors' : 'GSW',
        'Chicago Bulls' : 'CHI',
        'Cleveland Cavaliers' : 'CLE',
        'Indiana Pacers' : 'IND',
        'Phoenix Suns': 'PHO',
        'Los Angeles Clippers' : 'LAC',
        'Atlanta Hawks' : 'ATL',
        'Washington Bullets' : 'WSB',
        'New York Knicks' : 'NYK',
        'Seattle SuperSonics' : 'SEA',
        'Toronto Raptors' : 'TOR',
        'Miami Heat' : 'MIA',
        'Brooklyn Nets' : 'BRK',
        'Charlotte Hornets' : 'CHO',
        'Washington Wizards' : 'WAS',
        'Orlando Magic' : 'ORL',
        'Oklahoma City Thunder' : 'OKC',
        'Memphis Grizzlies' : 'MEM',
        'Sacramento Kings' : 'SAC',
        'New Orleans Pelicans' : 'NOP',
        'Minnesota Timberwolves' : 'MIN',
        'San Diego Clippers' : 'SDC',
        'Kansas City Kings' : 'KCK',
        'Charlotte Bobcats' : 'CHA',
        'Vancouver Grizzlies' : 'VAN',
        'New Orleans Hornets' : 'NOH',
        'New Orleans/Oklahoma City Hornets' : 'NOK'}

#Intervalos de anos dos dados das temporadas que serão extraidos
anos = list(range(1980,2021))

todos_jogadores = []
todos_mvps = []

## Funções para pegar os dados de mvps e os dados dos times

In [2]:
def pegar_mvps(url, dados_mvps):
    
    try:
        req = Request(url, headers = headers)

        response = urlopen(req)
        html = response.read()
        html = html.decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')

    except HTTPError as e:
        print(e.status, e.reason)

    except URLError as e:
        print(e.reason)
    
    mvps_html = soup.find('tbody').findAll('tr')
    cont_rank = 1
    for mvp_html in mvps_html:

        dados_mvp = {}
        
        dados_mvp['rank'] = cont_rank
        cont_rank += 1
        
        nome_jogador = mvp_html.find('td', {'data-stat':'player'}).find('a').getText()
        dados_mvp['team'] = mvp_html.find('td', {'data-stat':'team_id'}).getText()
        dados_mvp['award_share'] = float(mvp_html.find('td', {'data-stat':'award_share'}).getText())
        
        dados_mvps[nome_jogador] = dados_mvp


def pegar_times_conferencia(soup, dados_times, sufixo):
    times_html = soup.find('table', id = f'confs_standings_{sufixo}')
    
    #Em algumas páginas o id da tag difere, então estou fazendo essa pequena gambiarra para não dar problema,já que
    #só há dois padrões de id para a tag
    if times_html == None:
        times_html = soup.find('table', id = f'divs_standings_{sufixo}').find('tbody').findAll('tr')
    else:
        times_html = soup.find('table', id = f'confs_standings_{sufixo}').find('tbody').findAll('tr')
    
    
    for time_html in times_html:
        if(time_html.get('class')[0] != 'thead'):
            dados_time = {}
            nome_time = times[time_html.find('th').find('a').getText()]

            time_stats_html = time_html.findAll('td')

            for stat in time_stats_html:
                try:
                    dados_time[stat.get('data-stat')] = float(stat.getText())
                except ValueError:
                    dados_time[stat.get('data-stat')] = 0

            dados_times[nome_time] = dados_time

#Função apenas para fazer o request e gerar o objeto BeautifulSoup
def pegar_times(url,dados_times):
    try:
        req = Request(url_dados_time, headers = headers)

        response = urlopen(req)
        html = response.read()
        html = html.decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')

    except HTTPError as e:
        print(e.status, e.reason)

    except URLError as e:
        print(e.reason)
    
    print(ano)
    "Pegando os dados da Conferência Leste"
    pegar_times_conferencia(soup,dados_times, 'E')
    "Pegando os dados da Conferência Oeste"
    pegar_times_conferencia(soup,dados_times, 'W')
    

## Funções para carregar dados dos jogadores

In [3]:
def primeiro_scraping_jogadores(url, dados_jogadores, ano):
    try:
        req = Request(url, headers = headers)

        response = urlopen(req)
        html = response.read()
        html = html.decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')

    except HTTPError as e:
        print(e.status, e.reason)

    except URLError as e:
        print(e.reason)
    
    jogadores_html = soup.find('tbody').findAll('tr')
   
    for jogador_html in jogadores_html:
        dados_jogador = {}
        #As linhas da tabela que são da classe thead são meramente quebras entre as linhas,então não as considero
        if jogador_html.get('class')[0] != 'thead':
            
            #Novamente há dois padrões e faço essa pequena gambiarra
            if jogador_html.find('td', {'data-stat':'team_id'}).a == None:
                sigla_time = jogador_html.find('td', {'data-stat':'team_id'}).getText()
            else:
                sigla_time = jogador_html.find('td', {'data-stat':'team_id'}).a.getText()
            
            parametro_player = ''.join(jogador_html.find('td', {'data-stat':'player'}).find('a').getText().split(' '))
            dados_jogador['player_season_team'] = parametro_player +'_' + str(ano) + '_' + sigla_time
            dados_jogador['player'] = jogador_html.find('td', {'data-stat':'player'}).find('a').getText()
            dados_jogador['season'] = ano
            dados_jogador['pos'] = jogador_html.find('td', {'data-stat':'pos'}).getText()
            dados_jogador['age'] = int( jogador_html.find('td', {'data-stat':'age'}).getText() )
            dados_jogador['team'] = sigla_time
            
            for stat in jogador_html.findAll('td')[4:]:
                
                try:
                    dados_jogador[stat.get('data-stat')] = int(stat.getText())
                    
                except ValueError:
                    if(stat.getText() != ''):
                        dados_jogador[stat.get('data-stat')] = float('0' + stat.getText())
                    else:
                        dados_jogador[stat.get('data-stat')] = 0.0
            dados_jogador['mp_pergame'] = round(dados_jogador['mp']/dados_jogador['g'], 1)
            dados_jogadores.append(dados_jogador)

        
def intermedirario_scraping_jogadores(url, dados_jogadores):
    try:
        req = Request(url, headers = headers)

        response = urlopen(req)
        html = response.read()
        html = html.decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')

    except HTTPError as e:
        print(e.status, e.reason)

    except URLError as e:
        print(e.reason)
    
    jogadores_html = soup.find('tbody').findAll('tr')
    
    
    for jogador_html in jogadores_html:
        dados_jogador = dados_jogadores[i]
        
        if jogador_html.get('class')[0] != 'thead':
            
            stats_jogador = jogador_html.findAll('td')[8:]
            for stat in stats_jogador:
                try:
                    dados_jogador[stat.get('data-stat')] = int(stat.getText())

                except ValueError:
                    if(stat.getText() == ''):
                        dados_jogador[stat.get('data-stat')] = 0.0
                    elif stat.getText()[0] == '-':
                        dados_jogador[stat.get('data-stat')] =(-1)*float('0' + stat.getText()[1:])
                    else:
                        dados_jogador[stat.get('data-stat')] = float('0' + stat.getText())
                    
            

## Bloco principal da extração fazendo um loop por ano

In [4]:
for ano in anos:
    "Nesta seção do código serão carregadas as páginas html"
    
    url_dados_time = f'https://www.basketball-reference.com/leagues/NBA_{ano}_standings.html'
    url_dados_totais_jogador = f'https://www.basketball-reference.com/leagues/NBA_{ano}_totals.html'
    url_dados_por_jogo_jogador = f'https://www.basketball-reference.com/leagues/NBA_{ano}_per_game.html'
    url_dados_por_36min_jogador = f'https://www.basketball-reference.com/leagues/NBA_{ano}_per_minute.html'
    url_dados_por_100pos_jogador = f'https://www.basketball-reference.com/leagues/NBA_{ano}_per_poss.html'
    url_dados_avancados_jogador = f'https://www.basketball-reference.com/leagues/NBA_{ano}_advanced.html'
    url_votacao_mvp = f'https://www.basketball-reference.com/awards/awards_{ano}.html'
    
    
    "Aqui será carregado os dados dos candidatos a MVP relativos a este ano"
    
    dados_mvps = {}
    pegar_mvps(url_votacao_mvp,dados_mvps)
    
   
    
    "Aqui serão carregados os dados do time"
    
    "Aqui é a variável que armazenará os dados do time relativos àquele ano"
    dados_times = {}
    
    pegar_times(url_dados_time, dados_times)
    
    
    "Finalmente, serão carregados os dados dos jogadores"
    
    dados_jogadores = []
    
    #Pegando os dados de cada página
    
    primeiro_scraping_jogadores(url_dados_totais_jogador, dados_jogadores, int(ano))
    
    intermedirario_scraping_jogadores(url_dados_por_jogo_jogador, dados_jogadores)
    intermedirario_scraping_jogadores(url_dados_por_36min_jogador, dados_jogadores)
    intermedirario_scraping_jogadores(url_dados_por_100pos_jogador, dados_jogadores)
    intermedirario_scraping_jogadores(url_dados_avancados_jogador, dados_jogadores)
    
    #As chaves são justamente os nomes dos candidatos a MVP.Uso isso para ficar mais fácil verificar quem é candidato ou não
    nomes_dos_mvps = list(dados_mvps.keys())
    
    #Como o recorte é da década de 80 para cá, e o Lakers não mudou de nome desde esse ano, estou usando essa pequena gambiarra
    #para pegar as colunas dos times
    colunas_time = list(dados_times['LAL'].keys())
    
    for dados_jogador in dados_jogadores:
        sigla_time = dados_jogador['team']
        
        #TOT no basketball reference significa jogadores que jogaram em mais de um time.
        #Há pouquissimos jogadores nessa situação que foram candidatos a MVP e nenhum que ganhou o prêmio.
        if sigla_time != 'TOT': 
            
            if sigla_time == 'CHH':
                sigla_time = 'CHO'
            
            dict_time = dados_times[sigla_time]

            for coluna_time in colunas_time:
                
                dados_jogador['team_' + coluna_time] = dict_time[coluna_time]

            #Verifica se o jogador está entre os candidatos a MVP deste ano
            if dados_jogador['player'] in nomes_dos_mvps:
                dict_mvp = dados_mvps[dados_jogador['player']]
                    
                dados_jogador['mvp_candidate'] = True
                rank_mvp = dict_mvp['rank']


                if(rank_mvp<=4):
                    dados_jogador['mvp_podium'] = True
                    if(rank_mvp==1):
                        dados_jogador['mvp_winner'] = True
                    else:
                        dados_jogador['mvp_winner'] = False
                else:
                    dados_jogador['mvp_podium'] = False
                    dados_jogador['mvp_winner'] = False

                dados_jogador['mvp_votes_share'] = dict_mvp['award_share']

                parametro_player = ''.join(dados_jogador['player'].split(' '))

                parametro_plt = parametro_player + '_' + str(ano) + '_' + dados_jogador['team']
                todos_mvps.append({'player_season_team': parametro_plt,'season':ano ,'rank': rank_mvp,
                                   'mvp_votes_share': dict_mvp['award_share'], 'mvp_candidate':dados_jogador['mvp_candidate'],
                                   'mvp_podium':dados_jogador['mvp_podium'], 'mvp_winner':dados_jogador['mvp_winner']})

            else:
                dados_jogador['mvp_candidate'] = False
                dados_jogador['mvp_podium'] = False
                dados_jogador['mvp_winner'] = False
                dados_jogador['mvp_votes_share'] = 0.0
            

    todos_jogadores += dados_jogadores

1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


In [5]:
dataset1 = pd.DataFrame(todos_jogadores)
dataset1

,player_season_team,player,season,pos,age,team,g,gs,mp,fg,...,team_losses,team_win_loss_pct,team_gb,team_pts_per_g,team_opp_pts_per_g,team_srs,mvp_candidate,mvp_podium,mvp_winner,mvp_votes_share
0,KareemAbdul-Jabbar_1980_LAL,Kareem Abdul-Jabbar,1980,C,32,LAL,82,0.0,3143,835,...,22.0,0.732,0.0,115.1,109.2,5.40,True,True,True,0.665
1,TomAbernethy_1980_GSW,Tom Abernethy,1980,PF,25,GSW,67,0.0,1222,153,...,58.0,0.293,36.0,103.6,108.0,-4.20,False,False,False,0.000
2,AlvanAdams_1980_PHO,Alvan Adams,1980,C,25,PHO,75,0.0,2168,465,...,27.0,0.671,5.0,111.1,107.5,3.25,False,False,False,0.000
3,TinyArchibald_1980_BOS,Tiny Archibald,1980,PG,31,BOS,80,80.0,2864,383,...,21.0,0.744,0.0,113.5,105.7,7.37,True,False,False,0.009
4,DennisAwtrey_1980_CHI,Dennis Awtrey,1980,C,31,CHI,26,0.0,560,27,...,52.0,0.366,19.0,107.5,110.2,-2.63,False,False,False,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20945,TraeYoung_2020_ATL,Trae Young,2020,PG,21,ATL,60,60.0,2120,546,...,47.0,0.299,33.0,111.8,119.7,-7.71,False,False,False,0.000
20946,CodyZeller_2020_CHO,Cody Zeller,2020,C,27,CHO,58,39.0,1341,251,...,42.0,0.354,29.0,102.9,109.6,-7.03,False,False,False,0.000
20947,TylerZeller_2020_SAS,Tyler Zeller,2020,C,30,SAS,2,0.0,4,1,...,39.0,0.451,20.0,114.1,115.2,-0.65,False,False,False,0.000
20948,AnteŽižić_2020_CLE,Ante Žižić,2020,C,23,CLE,22,0.0,221,41,...,46.0,0.292,33.0,106.9,114.8,-7.77,False,False,False,0.000


In [6]:
dataset2 = pd.DataFrame(todos_mvps)
dataset2

,player_season_team,season,rank,mvp_votes_share,mvp_candidate,mvp_podium,mvp_winner
0,KareemAbdul-Jabbar_1980_LAL,1980,1,0.665,True,True,True
1,TinyArchibald_1980_BOS,1980,5,0.009,True,False,False
2,LarryBird_1980_BOS,1980,4,0.068,True,True,False
3,JuliusErving_1980_PHI,1980,2,0.143,True,True,False
4,GeorgeGervin_1980_SAS,1980,3,0.086,True,True,False
...,...,...,...,...,...,...,...
671,KawhiLeonard_2020_LAC,2020,5,0.166,True,False,False
672,DamianLillard_2020_POR,2020,8,0.023,True,False,False
673,ChrisPaul_2020_OKC,2020,7,0.026,True,False,False
674,PascalSiakam_2020_TOR,2020,10,0.017,True,False,False


In [7]:
dataset1.to_csv('./data/dataset_jogadores.csv', sep=';', index = False, encoding = 'utf-8-sig')
dataset2.to_csv('./data/dataset_mvps.csv', sep=';', index = False, encoding = 'utf-8-sig')

## Fazendo um tratamento básico

In [8]:
import pandas as pd

dataset_jogadores = pd.read_csv('./data/dataset_jogadores.csv', sep=';')

dataset_jogadores.head()

,player_season_team,player,season,pos,age,team,g,gs,mp,fg,...,team_losses,team_win_loss_pct,team_gb,team_pts_per_g,team_opp_pts_per_g,team_srs,mvp_candidate,mvp_podium,mvp_winner,mvp_votes_share
0,KareemAbdul-Jabbar_1980_LAL,Kareem Abdul-Jabbar,1980,C,32,LAL,82,0.0,3143,835,...,22.0,0.732,0.0,115.1,109.2,5.40,True,True,True,0.665
1,TomAbernethy_1980_GSW,Tom Abernethy,1980,PF,25,GSW,67,0.0,1222,153,...,58.0,0.293,36.0,103.6,108.0,-4.20,False,False,False,0.000
2,AlvanAdams_1980_PHO,Alvan Adams,1980,C,25,PHO,75,0.0,2168,465,...,27.0,0.671,5.0,111.1,107.5,3.25,False,False,False,0.000
3,TinyArchibald_1980_BOS,Tiny Archibald,1980,PG,31,BOS,80,80.0,2864,383,...,21.0,0.744,0.0,113.5,105.7,7.37,True,False,False,0.009
4,DennisAwtrey_1980_CHI,Dennis Awtrey,1980,C,31,CHI,26,0.0,560,27,...,52.0,0.366,19.0,107.5,110.2,-2.63,False,False,False,0.000


In [9]:
for coluna in dataset_jogadores.columns:
    print(coluna)

player_season_team
player
season
pos
age
team
g
gs
mp
fg
fga
fg_pct
fg3
fg3a
fg3_pct
fg2
fg2a
fg2_pct
efg_pct
ft
fta
ft_pct
orb
drb
trb
ast
stl
blk
tov
pf
pts
mp_pergame
fga_per_g
fg3_per_g
fg3a_per_g
fg2_per_g
fg2a_per_g
ft_per_g
fta_per_g
orb_per_g
drb_per_g
trb_per_g
ast_per_g
stl_per_g
blk_per_g
tov_per_g
pf_per_g
pts_per_g
fga_per_mp
fg3_per_mp
fg3a_per_mp
fg2_per_mp
fg2a_per_mp
ft_per_mp
fta_per_mp
orb_per_mp
drb_per_mp
trb_per_mp
ast_per_mp
stl_per_mp
blk_per_mp
tov_per_mp
pf_per_mp
pts_per_mp
fga_per_poss
fg3_per_poss
fg3a_per_poss
fg2_per_poss
fg2a_per_poss
ft_per_poss
fta_per_poss
orb_per_poss
drb_per_poss
trb_per_poss
ast_per_poss
stl_per_poss
blk_per_poss
tov_per_poss
pf_per_poss
pts_per_poss
Unnamed: 80
off_rtg
def_rtg
fg3a_per_fga_pct
fta_per_fga_pct
orb_pct
drb_pct
trb_pct
ast_pct
stl_pct
blk_pct
tov_pct
usg_pct
DUMMY
ows
dws
ws
ws_per_48
obpm
dbpm
bpm
vorp
team_wins
team_losses
team_win_loss_pct
team_gb
team_pts_per_g
team_opp_pts_per_g
team_srs
mvp_candidate
mvp_podium

### As duas colunas abaixo  são de colunas vazias do basketball reference, então elas devem ser apagadas

In [10]:
dataset_jogadores['Unnamed: 80']

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
20945    0.0
20946    0.0
20947    0.0
20948    0.0
20949    0.0
Name: Unnamed: 80, Length: 20950, dtype: float64

In [11]:
dataset_jogadores['DUMMY']

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
20945    0.0
20946    0.0
20947    0.0
20948    0.0
20949    0.0
Name: DUMMY, Length: 20950, dtype: float64

In [12]:
dataset_jogadores.drop(columns = ['Unnamed: 80', 'DUMMY'], inplace = True)
dataset_jogadores.head()

,player_season_team,player,season,pos,age,team,g,gs,mp,fg,...,team_losses,team_win_loss_pct,team_gb,team_pts_per_g,team_opp_pts_per_g,team_srs,mvp_candidate,mvp_podium,mvp_winner,mvp_votes_share
0,KareemAbdul-Jabbar_1980_LAL,Kareem Abdul-Jabbar,1980,C,32,LAL,82,0.0,3143,835,...,22.0,0.732,0.0,115.1,109.2,5.40,True,True,True,0.665
1,TomAbernethy_1980_GSW,Tom Abernethy,1980,PF,25,GSW,67,0.0,1222,153,...,58.0,0.293,36.0,103.6,108.0,-4.20,False,False,False,0.000
2,AlvanAdams_1980_PHO,Alvan Adams,1980,C,25,PHO,75,0.0,2168,465,...,27.0,0.671,5.0,111.1,107.5,3.25,False,False,False,0.000
3,TinyArchibald_1980_BOS,Tiny Archibald,1980,PG,31,BOS,80,80.0,2864,383,...,21.0,0.744,0.0,113.5,105.7,7.37,True,False,False,0.009
4,DennisAwtrey_1980_CHI,Dennis Awtrey,1980,C,31,CHI,26,0.0,560,27,...,52.0,0.366,19.0,107.5,110.2,-2.63,False,False,False,0.000


## Salvando após as duas remoções

In [13]:
dataset_jogadores.to_csv('./data/dataset_jogadores.csv', sep=';', index = False, encoding = 'utf-8-sig')

## Cenas para os próximos capitulos

In [14]:
#Seleção de jogadores que jogaram menos de 20 min por média e foram candidatos a MVP
selecao = (dataset_jogadores['mvp_candidate'] == True) & (dataset_jogadores['mp_pergame'] < 20)
dataset_jogadores[selecao]

,player_season_team,player,season,pos,age,team,g,gs,mp,fg,...,team_losses,team_win_loss_pct,team_gb,team_pts_per_g,team_opp_pts_per_g,team_srs,mvp_candidate,mvp_podium,mvp_winner,mvp_votes_share


In [15]:
#Seleção de jogadores que jogaram menos de 40% das partidas da temporada e foram candidatos a MVP
selecao = (dataset_jogadores['mvp_candidate'] == True) & (dataset_jogadores['g']/(dataset_jogadores['team_wins'] + dataset_jogadores['team_losses']) < 0.4)
dataset_jogadores[selecao]

,player_season_team,player,season,pos,age,team,g,gs,mp,fg,...,team_losses,team_win_loss_pct,team_gb,team_pts_per_g,team_opp_pts_per_g,team_srs,mvp_candidate,mvp_podium,mvp_winner,mvp_votes_share
6069,DominiqueWilkins_1994_LAC,Dominique Wilkins,1994,SF,34,LAC,25,25.0,948,268,...,55.0,0.329,36.0,103.0,108.7,-5.28,True,False,False,0.001
6306,MichaelJordan_1995_CHI,Michael Jordan,1995,SG,31,CHI,17,17.0,668,166,...,35.0,0.573,5.0,101.5,96.7,4.32,True,False,False,0.011
6781,MagicJohnson_1996_LAL,Magic Johnson,1996,PF,36,LAL,32,9.0,958,137,...,29.0,0.646,11.0,102.9,98.5,4.21,True,False,False,0.007
11426,VinceCarter_2005_TOR,Vince Carter,2005,SG,28,TOR,20,20.0,608,124,...,49.0,0.402,12.0,99.7,101.4,-1.81,True,False,False,0.002
13638,ChaunceyBillups_2009_DET,Chauncey Billups,2009,PG,32,DET,2,2.0,70,7,...,43.0,0.476,27.0,94.2,94.7,-0.36,True,False,False,0.027
14448,StephenJackson_2010_GSW,Stephen Jackson,2010,PF,31,GSW,9,9.0,300,56,...,56.0,0.317,31.0,108.8,112.4,-3.28,True,False,False,0.001


In [16]:
#Seleção de jogadores que jogaram em times com %wins_losses menor que 40% na temporada e foram candidatos a MVP
selecao = (dataset_jogadores['mvp_candidate'] == True) & (dataset_jogadores['team_win_loss_pct'] < 0.39)
dataset_jogadores[selecao]

,player_season_team,player,season,pos,age,team,g,gs,mp,fg,...,team_losses,team_win_loss_pct,team_gb,team_pts_per_g,team_opp_pts_per_g,team_srs,mvp_candidate,mvp_podium,mvp_winner,mvp_votes_share
424,AdrianDantley_1981_UTA,Adrian Dantley,1981,SF,25,UTA,80,0.0,3417,909,...,54.0,0.341,24.0,101.2,107.1,-5.99,True,False,False,0.022
594,MikeMitchell_1981_CLE,Mike Mitchell,1981,SF,25,CLE,82,0.0,3194,853,...,54.0,0.341,32.0,105.7,110.6,-4.15,True,False,False,0.004
792,AdrianDantley_1982_UTA,Adrian Dantley,1982,SF,26,UTA,81,81.0,3222,904,...,57.0,0.305,23.0,110.9,116.6,-5.63,True,False,False,0.010
1054,JayVincent_1982_DAL,Jay Vincent,1982,PF,22,DAL,81,62.0,2626,719,...,54.0,0.341,20.0,104.6,109.0,-4.48,True,False,False,0.001
1162,TerryCummings_1983_SDC,Terry Cummings,1983,PF,21,SDC,70,69.0,2531,684,...,57.0,0.305,33.0,108.6,113.4,-4.61,True,False,False,0.013
1575,WorldB.Free_1984_CLE,World B. Free,1984,SG,30,CLE,75,71.0,2375,626,...,54.0,0.341,22.0,102.3,106.5,-3.71,True,False,False,0.001
1990,BernardKing_1985_NYK,Bernard King,1985,SF,28,NYK,55,55.0,2063,691,...,58.0,0.293,39.0,105.2,109.8,-4.09,True,False,False,0.090
2099,PurvisShort_1985_GSW,Purvis Short,1985,SF,27,GSW,78,77.0,3081,819,...,60.0,0.268,40.0,110.4,117.7,-7.21,True,False,False,0.003
2101,JackSikma_1985_SEA,Jack Sikma,1985,C,29,SEA,68,68.0,2402,461,...,51.0,0.378,31.0,102.1,107.6,-5.44,True,False,False,0.004
3348,BuckWilliams_1988_NJN,Buck Williams,1988,PF,27,NJN,70,70.0,2637,466,...,63.0,0.232,38.0,100.4,108.5,-6.98,True,False,False,0.001
